In [89]:
import json
from cogent3 import get_app, open_data_store
from cogent3 import make_tree
from cogent3.app.composable import NotCompleted
from mdeq.utils import estimate_freq_null
import json

load_json_app = get_app("load_json")



def get_param_rules_upper_limit(model_name, upper):
    """rules to set the upper value for rate matrix terms"""
    from cogent3 import get_model

    sm = get_model(model_name)
    return [{"par_name": par_name, "upper": upper} for par_name in sm.get_param_list()]

def p_value(result):
    return sum(result.observed.LR <= null_lr for null_lr in result.null_dist) / len(result.null_dist)

def p_value2(result, value):
    return sum(value <= null_lr for null_lr in result.null_dist) / len(result.null_dist)

## Test of non-stationary

Non-stationarity proportion from bootstrape

In [90]:
ton_result_dir = '/Users/gulugulu/Desktop/honours/data_local_2/bootstrapping_test_non'

input_data_store_ton_result = open_data_store(ton_result_dir, suffix= 'json')

p_value_observed_dict = {}
p_value_tested_dict = {}
i = 0
for data in input_data_store_ton_result:
    gene_name = data.unique_id.split('.')[0]
    bootstrap_result = load_json_app(data)
    if isinstance(bootstrap_result, NotCompleted):
        i += 1
        print(data) 
    else:
        p_value_observed_dict[gene_name] = bootstrap_result.observed.pvalue
        p_value_tested_dict[gene_name] = p_value(bootstrap_result)

# get the p-value of null simulated for each algnment 
from random import sample
#get distirbution of null for ton
p_value_list_ton_null = {}
for path in input_data_store_ton_result:
    gene_name = path.unique_id.split('.')[0]
    bootstrap_result = load_json_app(path)
    if not isinstance(bootstrap_result, NotCompleted):
        pvalue = None
        # Keep sampling until a non-None pvalue is found
        while pvalue is None:
            null_sample_key = sample(list(bootstrap_result.keys())[1:], 1)  # Sampling from keys, skip first key
            sub_result = bootstrap_result[null_sample_key[0]]
            pvalue = sub_result.pvalue  # Get the pvalue from the sub_result
        p_value = p_value2(bootstrap_result, sub_result.LR)
        p_value_list_ton_null[gene_name] = (p_value)


ENSG00000198730_435.json
ENSG00000166106_357.json
ENSG00000173662_152.json
ENSG00000158966_36.json
ENSG00000148700_465.json
ENSG00000133816_93.json
ENSG00000187554_73.json
ENSG00000138193_276.json
ENSG00000122483_416.json


In [131]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import uniform

def qq_plot_uniform(data, a=0, b=1):
    """
    Creates a QQ plot of the provided data against a uniform distribution using Plotly.

    Args:
    data (array-like): The dataset to plot.
    a (float): The lower bound of the uniform distribution (default 0).
    b (float): The upper bound of the uniform distribution (default 1).
    """
    # Scale data for the specified uniform range
    data = np.array(data)
    data.sort()  # Sort the data for plotting
    scaled_data = (data - a) / (b - a)

    # Calculate quantiles
    n = len(data)
    theoretical_quantiles = uniform.ppf(np.arange(1, n + 1) / (n + 1))

    # Create a QQ plot
    fig = go.Figure()

    # Adding scatter plot for QQ plot
    fig.add_trace(go.Scatter(x=theoretical_quantiles, y=scaled_data, mode='markers',
                             name=None,
                             marker=dict(color='blue', size = 5)))

    # Add line of perfect fit
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
                             name=None,
                             line=dict(color='red', dash='dash')))

    # Update layout
    fig.update_layout(
                      xaxis_title='Uniform Quantiles',
                      yaxis_title='p-value',
                      width=600, height=600,
                      showlegend=False)

    return fig


def qq_plot_null_observed(data, data2, a=0, b=1):
    """
    Creates a QQ plot of the provided data against a uniform distribution using Plotly.

    Args:
    data (array-like): The dataset to plot.
    a (float): The lower bound of the uniform distribution (default 0).
    b (float): The upper bound of the uniform distribution (default 1).
    """
    # Scale data for the specified uniform range
    data = np.array(data)
    data.sort()  # Sort the data for plotting
    scaled_data = (data - a) / (b - a)

    # Calculate quantiles
    n = len(data)
    theoretical_quantiles = uniform.ppf(np.arange(1, n + 1) / (n + 1))

    # Scale data for the specified uniform range
    data2 = np.array(data2)
    data2.sort()  # Sort the data for plotting
    scaled_data2 = (data2 - a) / (b - a)

    # Calculate quantiles
    n2 = len(data2)
    theoretical_quantiles2 = uniform.ppf(np.arange(1, n2 + 1) / (n2 + 1))

    # Create a QQ plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=theoretical_quantiles, y=scaled_data, mode='markers',
                                name='Null',
                                marker=dict(color='blue', size = 2)))

    # Adding scatter plot for QQ plot
    fig.add_trace(go.Scatter(x=theoretical_quantiles2, y=scaled_data2, mode='markers',
                                name='Observed',
                                marker=dict(color='green', size = 2)))

    # # Add line of perfect fit
    # fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
    #                             name=None,
    #                             line=dict(color='red', dash='dash')))

    # Update layout
    fig.update_layout(
                        xaxis_title='Uniform Quantiles',
                        yaxis_title='p-value',
                        width=600, height=600)
    
    return fig


In [4]:
with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/genome_information/p_value_dict_observed_ton.json', 'r') as infile1:
    p_value_tested_dict = json.load(infile1)

In [5]:
with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/genome_information/p_value_list_simulated_null_ton.json', 'r') as infile:
    p_value_list_ton_null = json.load(infile)

In [132]:
fig = qq_plot_null_observed(list(p_value_list_ton_null.values()), list(p_value_tested_dict.values()))
fig.update_layout(title='Test of non-stationarity - P-value distribution', xaxis_title='Uniform Quantiles', yaxis_title='p-value',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 20},  
                    xaxis_title_font={'size': 20}, 
                    width=None)
fig.show()

In [7]:
# example_p_value_list_non_uniform_null = []
# example_ton_bootstrape_result = load_json_app(input_data_store_ton_result[4])
# for key in example_ton_bootstrape_result.keys():
#     if key != 'observed':
#         example_p_value_list_non_uniform_null.append(example_ton_bootstrape_result[key].pvalue)


In [119]:
with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/genome_information/example_non_uniform_null_p_value_ton.json', 'r') as infile3:
    example_p_value_list_non_uniform_null = json.load(infile3)

fig = qq_plot_uniform([p for p in example_p_value_list_non_uniform_null if p != None])

fig.update_layout(title='Test of non-stationarity - P-value distribution of NULL', 
                  xaxis_title='Uniform Quantiles', 
                  yaxis_title='p-value',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 20},  
                    xaxis_title_font={'size': 20}, 
                    width=None)
fig.show()

In [109]:
from collections import defaultdict
# Dictionary to store the count of values < 0.05 for each gene
count_less_than_005_non = defaultdict(int)
# Dictionary to store the total count of entries for each gene
total_count = defaultdict(int)

for key, value in p_value_tested_dict.items():
    gene = key.split('_')[0]
    total_count[gene] += 1
    if value < 0.05:
        count_less_than_005_non[gene] += 1

# Calculate the proportion of values < 0.05 for each gene
proportion_less_than_005_non = {gene: count_less_than_005_non[gene] / total for gene, total in total_count.items()}

count_gene = len([x for x in proportion_less_than_005_non.values() if x > 0.29])

count_gene #73%, 43.8%

95

In [11]:
gene_list = [gene for gene in proportion_less_than_005_non.keys() if proportion_less_than_005_non[gene] > 0.29]

In [12]:
corr_list = {'ENSG00000137962': 0.4212773464208629,
 'ENSG00000122482': 0.813500142616243,
 'ENSG00000116688': 0.19250352361797127,
 'ENSG00000122218': 0.021225367220316132,
 'ENSG00000054523': 0.13901530575091464,
 'ENSG00000030066': 0.10145033156765873,
 'ENSG00000177888': 0.3368146543979869,
 'ENSG00000117707': 0.14339147597059745,
 'ENSG00000122483': 0.3019081470902082,
 'ENSG00000138119': 0.34772571287228177,
 'ENSG00000107611': 0.1490774112859659,
 'ENSG00000149256': 0.1868882727898733,
 'ENSG00000085999': 0.5240505399815937,
 'ENSG00000187122': 0.35677330589019385,
 'ENSG00000135372': 0.27822223665464235,
 'ENSG00000197147': 0.7919380919967495,
 'ENSG00000148948': -0.1155196504527771,
 'ENSG00000058668': 0.28469336080525853,
 'ENSG00000116991': 0.22142782037503286,
 'ENSG00000165494': -0.016980295912680875,
 'ENSG00000066135': -0.004524514554078555,
 'ENSG00000174576': 0.2025439958145585,
 'ENSG00000148516': 0.6328793045678983,
 'ENSG00000163399': 0.50895402794267,
 'ENSG00000129173': 0.6970256278751281,
 'ENSG00000110237': 0.17733401166901053,
 'ENSG00000185085': 0.09285317982345617,
 'ENSG00000162688': 0.8773677742444248,
 'ENSG00000133816': 0.07201792082078974,
 'ENSG00000151461': 0.4596610666052429,
 'ENSG00000117425': 0.08977090937550986,
 'ENSG00000260314': 0.7501472288279593,
 'ENSG00000197106': 0.24526679576598748,
 'ENSG00000143126': -0.005711685562393361,
 'ENSG00000095587': 0.3275915034006169,
 'ENSG00000058085': 0.05218581078391571,
 'ENSG00000187554': 0.42760603651235995,
 'ENSG00000143322': 0.2961139255277253,
 'ENSG00000265203': 0.21835891206067434,
 'ENSG00000110075': 0.2690084404804709,
 'ENSG00000016490': 0.556800297308761,
 'ENSG00000171488': 0.7388372273069503,
 'ENSG00000148700': 0.7814316608368127,
 'ENSG00000142661': 0.14925444875238833,
 'ENSG00000196878': 0.2309959691109984,
 'ENSG00000117000': 0.8610030483735327,
 'ENSG00000143776': 0.635634448679333,
 'ENSG00000116128': 0.3140603802412976,
 'ENSG00000117724': -0.014335415789467204,
 'ENSG00000048707': 0.48117367040195574,
 'ENSG00000118193': 0.8157819014138401,
 'ENSG00000166341': 0.38920979764536484,
 'ENSG00000117322': 0.34644275440527783,
 'ENSG00000110756': 0.8046981197941429,
 'ENSG00000065613': 0.9052334134639237,
 'ENSG00000138193': 0.4257083009540453,
 'ENSG00000009307': 0.5307702563556597,
 'ENSG00000129083': 0.6233085160460541,
 'ENSG00000095464': 0.44378995788994446,
 'ENSG00000148843': 0.06787614395900958,
 'ENSG00000166349': 0.11933943853815006,
 'ENSG00000117139': 0.5901843437666873,
 'ENSG00000143442': 0.12480131209547311,
 'ENSG00000081248': 0.18424916462650892,
 'ENSG00000196914': 0.6567113267004492,
 'ENSG00000052841': 0.5068957159059464,
 'ENSG00000138182': 0.6172094142256217,
 'ENSG00000158636': 0.5937826197640631,
 'ENSG00000166106': 0.35048475775906157,
 'ENSG00000160695': -0.1046738650243683,
 'ENSG00000198216': 0.37401776301522044,
 'ENSG00000166507': -0.0436760340094065,
 'ENSG00000051009': 0.47743511020769264,
 'ENSG00000116539': 0.4128890994101011,
 'ENSG00000020129': 0.5245736496734354,
 'ENSG00000198952': 0.1304477045564663,
 'ENSG00000150347': 0.2513970067194448,
 'ENSG00000110497': 0.7134183699351087,
 'ENSG00000198561': 0.7990077455272536,
 'ENSG00000118058': 0.6079068425646565,
 'ENSG00000134250': 0.5064974206901268,
 'ENSG00000109927': 0.16724788630567028,
 'ENSG00000076356': 0.5329366047548466,
 'ENSG00000143702': 0.5519656989212849,
 'ENSG00000126705': 0.23228307047177604,
 'ENSG00000118200': 0.6823309908326792,
 'ENSG00000257315': 0.29297765536698084,
 'ENSG00000198730': 0.5901960427342248,
 'ENSG00000137474': 0.3300668366616139,
 'ENSG00000065526': 0.5194483896969769,
 'ENSG00000047410': 0.749395000923836,
 'ENSG00000204624': 0.17347169411855284,
 'ENSG00000095564': 0.5276082609903987,
 'ENSG00000126107': 0.3049791810256095,
 'ENSG00000214655': 0.22175012466449653,
 'ENSG00000107863': 0.6045815136762446,
 'ENSG00000167986': 0.500394731043054,
 'ENSG00000175216': 0.12924065910528748,
 'ENSG00000075151': 0.2256629243496099,
 'ENSG00000133056': 0.2124867428718397,
 'ENSG00000152092': 0.18909052869397283,
 'ENSG00000117600': 0.5751434776084783,
 'ENSG00000110422': 0.7941150911119135,
 'ENSG00000116183': 0.2768544394417856,
 'ENSG00000154309': 0.5330308250296808,
 'ENSG00000158966': 0.5538185103553634,
 'ENSG00000169641': 0.24561278534481365,
 'ENSG00000121753': 0.3392024443365115,
 'ENSG00000133059': 0.01780769867391686,
 'ENSG00000107862': 0.2607715631658015,
 'ENSG00000162402': 0.28055863099085887,
 'ENSG00000075391': 0.29788110072648527,
 'ENSG00000042781': 0.607802574711679,
 'ENSG00000116147': 0.2709718262049579,
 'ENSG00000111642': 0.3602821728418383,
 'ENSG00000236287': 0.25318618515071944,
 'ENSG00000064309': 0.4383068588203169,
 'ENSG00000117114': 0.8583963017232994,
 'ENSG00000162711': 0.40969510288920535,
 'ENSG00000110799': 0.16229243434705412,
 'ENSG00000127124': 0.3364030802586702,
 'ENSG00000170242': 0.9210935106101873,
 'ENSG00000173898': 0.09725019044000538,
 'ENSG00000143493': 0.8656706368262804,
 'ENSG00000173662': 0.0037113557994933556,
 'ENSG00000162775': -0.0158981604986629,
 'ENSG00000023839': 0.06797707108880507,
 'ENSG00000135829': 0.3615104081567422,
 'ENSG00000177853': 0.3403120965494682,
 'ENSG00000176986': 0.6703812068917575}

In [13]:
sorted_corr = {gene: corr_list[gene] for gene in proportion_less_than_005_non.keys()}

In [14]:
import plotly.express as px
fig = px.scatter(x = list(proportion_less_than_005_non.values()), y = list(sorted_corr.values()), labels={'x':'Non-stationary Evolution propotion', 'y':'Correlation Coefficient'}, trendline="ols", title= None)
# Update layout with labels and title
fig.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    width=None 
)
fig.show()

In [15]:
from scipy.stats import pearsonr
p, corr = pearsonr(list(proportion_less_than_005.values()), list(sorted_corr.values()))
p, corr

(0.36107087061421556, 2.4391516130253273e-05)

In [16]:
import numpy as np
from scipy.stats import entropy

def get_entropy_difference(disitrbution_list):

    # Convert to numpy array for easier manipulation
    distributions = np.array(disitrbution_list)

    # Step 1: Calculate the entropy of each individual distribution
    individual_entropies = np.array([entropy(d) for d in distributions])
    # print(f"Individual Entropies: {individual_entropies}")

    # Step 2: Calculate the average distribution
    average_distribution = np.mean(distributions, axis=0)
    # print(f"Average Distribution: {average_distribution}")

    # Step 3: Calculate the entropy of the average distribution
    average_entropy = entropy(average_distribution)
    # print(f"Entropy of Average Distribution: {average_entropy}")

    # Step 4: Calculate the average of the individual entropies
    mean_individual_entropy = np.mean(individual_entropies)
    # print(f"Mean of Individual Entropies: {mean_individual_entropy}")

    # Step 5: Compute the difference
    entropy_difference = average_entropy - mean_individual_entropy
    # print(f"Entropy Difference (Variability Measure): {entropy_difference}")

    return entropy_difference


In [17]:
from collections import defaultdict
p_value_dict_by_gene = {}
count_less_than_005 = defaultdict(int)
# Dictionary to store the total count of entries for each gene
total_count = defaultdict(int)

for key, value in p_value_tested_dict.items():
    gene = key.split('_')[0]
    if gene not in p_value_dict_by_gene:
        p_value_dict_by_gene[gene] = []
        p_value_dict_by_gene[gene].append(value)
    else:
        p_value_dict_by_gene[gene].append(value)
#     total_count[gene] += 1
#     if value < 0.05:
#         count_less_than_005[gene] += 1

# # Calculate the proportion of values < 0.05 for each gene
# proportion_less_than_005 = {gene: count_less_than_005[gene] / total for gene, total in total_count.items()}

# count_gene = len([x for x in proportion_less_than_005.values() if x > 0.29])

# count_gene 

estimated_non_null_prop = {}
for gene, p_values in p_value_dict_by_gene.items():
    try:
        estimated_non_null_prop[gene] = 1-estimate_freq_null(np.array(p_values))
    except IndexError as e:
        print(gene)

ENSG00000162402


In [88]:
# import plotly.express as px
# proportion1 = {gene: proportion_less_than_005[gene] for gene in estimated_non_null_prop.keys()}
# corr1 = {gene: corr_list[gene] for gene in estimated_non_null_prop.keys()}
# corr0 = {gene: corr_list[gene] for gene in proportion_less_than_005.keys()}
# fig = px.scatter(x = list(proportion1.values()), y = list(estimated_non_null_prop.values()), labels={'x': 'Propotion of significant', 'y': 'Proportion of NOT null'}, title= None)
# # Update layout with labels and title
# fig.update_layout(
#     template='plotly_white',
#     margin=dict(l=20, r=20, t=50, b=20),
#     autosize=True,
#     yaxis_title_font={'size': 20},  
#     xaxis_title_font={'size': 20}, 
#     width=None 
# )
# fig.show()



In [19]:
import plotly.express as px
fig = px.scatter(x = list(estimated_non_null_prop.values()), y = list(corr1.values()), labels={'x':'Proportion of Significance', 'y':'Correlation Coefficient'}, trendline="ols", title= None)
# Update layout with labels and title
fig.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    width=None 
)
fig.show()

In [20]:
from scipy.stats import pearsonr
p, corr = pearsonr(list(estimated_non_null_prop.values()), list(corr1.values()))
p, corr

(0.29029727642975056, 0.0008462164978035779)

## Test of Molecular clock Violaiton

In [91]:
from cogent3 import get_app, load_aligned_seqs
from cogent3.app import evo
load_json_app = get_app("load_json")

aln_dir = '/Users/gulugulu/Desktop/honours/data_local_2/triples_aln_representative_subset'
aln_input_data_store = open_data_store(aln_dir, suffix='json', mode='r')


def test_clock(aln):
    tree =  make_tree(tip_names=aln.names)
    sp1 = aln.info['triples_species_name']['ingroup1']
    sp2 = aln.info['triples_species_name']['ingroup2']

    outgroup_name = aln.info['triples_species_name']['outgroup']
    print(outgroup_name)
    outgroup_edge = [outgroup_name]

    model_kwargs = dict(
    tree=tree,
    opt_args=None,
    # unique_trees=True,
    lf_args=dict(discrete_edges=[outgroup_edge]),
    optimise_motif_probs=True,
    )

    null = get_app("model", "GN", name="clock", param_rules=[dict(par_name="length", edges=[sp1, sp2], is_independent=False)], **model_kwargs)
    alt = get_app("model", "GN", name="no-clock", **model_kwargs)
    hyp = get_app("hypothesis", null, alt)

    # clock_p_value_observed[gene_name] = (result.pvalue)
    result_test = hyp(aln)

    return result_test    



In [75]:
aln_dir = '/Users/gulugulu/Desktop/honours/data_local_2/triples_aln_representative_subset'
aln_input_data_store = open_data_store(aln_dir, suffix='json', mode='r')
aln = load_json_app(aln_input_data_store[0])
example_result = test_clock(aln)

Panamanian_white_faced_capuchin


In [92]:
clock_test_result = {}
p_value_dict_observed_aln_clock = {}
aln_dir = '/Users/gulugulu/Desktop/honours/data_local_2/triples_aln_representative_subset'
aln_input_data_store = open_data_store(aln_dir, suffix='json', mode='r')
for path in aln_input_data_store:
    gene_name = path.unique_id.split('.')[0]
    print(gene_name)
    aln = load_json_app(path)
    result = test_clock(aln)
    clock_test_result[gene_name] = result
    p_value_dict_observed_aln_clock[gene_name] = result.pvalue

ENSG00000065526_323
Panamanian_white_faced_capuchin
ENSG00000009307_3
Pig
ENSG00000126705_400
Armadillo
ENSG00000137962_134
Elephant
ENSG00000065526_266
Upper_Galilee_mountains_blind_mole_rat
ENSG00000110756_475
Arabian_camel
ENSG00000171488_299
Microbat
ENSG00000066135_207
Elephant
ENSG00000118200_101
Elephant
ENSG00000052841_164
Greater_horseshoe_bat
ENSG00000126705_32
Elephant
ENSG00000117707_480
Rabbit
ENSG00000117724_244
Microbat
ENSG00000138119_57
Microbat
ENSG00000110799_46
Elephant
ENSG00000143493_358
Armadillo
ENSG00000117707_195
Elephant
ENSG00000076356_455
Elephant
ENSG00000133816_38
Eurasian_red_squirrel
ENSG00000118058_245
Dog
ENSG00000167986_101
Bolivian_squirrel_monkey
ENSG00000116183_189
Armadillo
ENSG00000170242_129
Microbat
ENSG00000198730_58
Armadillo
ENSG00000148948_19
Ma's_night_monkey
ENSG00000154309_421
Armadillo
ENSG00000075391_127
Long_tailed_chinchilla
ENSG00000117707_341
Tarsier
ENSG00000116147_338
Armadillo
ENSG00000148516_124
Elephant
ENSG00000048707_310
Ho

In [23]:
with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/genome_information/p_value_dict_observed_aln_clock.json', 'r') as infile7:
    p_value_dict_observed_aln_clock = json.load(infile7)

In [24]:
def boostrap_clock(aln):
    tree =  make_tree(tip_names=aln.names)
    sp1 = aln.info['triples_species_name']['ingroup1']
    sp2 = aln.info['triples_species_name']['ingroup2']

    outgroup_name = aln.info['triples_species_name']['outgroup']
    print(outgroup_name)
    outgroup_edge = [outgroup_name]

    model_kwargs = dict(
    tree=tree,
    opt_args=None,
    # unique_trees=True,
    lf_args=dict(discrete_edges=[outgroup_edge]),
    optimise_motif_probs=True,
    )

    null = get_app("model", "GN", name="clock", param_rules=[dict(par_name="length", edges=[sp1, sp2], is_independent=False)], **model_kwargs)
    alt = get_app("model", "GN", name="no-clock", **model_kwargs)
    hyp = get_app("hypothesis", null, alt)

    # clock_p_value_observed[gene_name] = (result.pvalue)
    bootstrapper = get_app("bootstrap", hyp, num_reps=100, parallel=False)

    return bootstrapper(aln)

In [115]:
with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/genome_information/example_uniform_p_value_null_clock_test.json', 'r') as infile4:
    p_value_clock_unifor_null_list = json.load(infile4)

fig = qq_plot_uniform([p for p in p_value_clock_unifor_null_list if p != None])
fig.update_layout(title='Test of clock violation - P-value distribution of NULL', 
                   xaxis_title='Uniform Quantiles', 
                   yaxis_title='p-value',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 20},  
                    xaxis_title_font={'size': 20}, 
                    width=None)
fig.show()


get simulated null aln for clock test

In [26]:
# simulated_aln_list = []
# for result in clock_test_result:
#     if not isinstance(result, NotCompleted):
#         aln = result.null.alignment
#         triples_species_name = aln.info['triples_species_name']
#         null_res = result.null
#         simulated_aln = null_res.simulate_alignment()
#         simulated_aln.info['triples_species_name'] = triples_species_name
#         simulated_aln.info.source = aln.info.source
#         simulated_aln_list.append(simulated_aln)

In [27]:
# p_value_list_simulated_aln_clock = []
# result_simulated_aln = []
# for aln in simulated_aln_list:
#     result = test_clock(aln)
#     p_value_list_simulated_aln_clock.append(result.pvalue)
#     result_simulated_aln.append(result)

In [126]:
with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/genome_information/p_value_list_simulated_aln_clock.json', 'r') as infile5:
    p_value_list_simulated_aln_clock = json.load(infile5)


with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/genome_information/p_value_list_observed_aln_clock.json', 'r') as infile6:
    p_value_list_observed_aln_clock = json.load(infile6)

fig = qq_plot_null_observed([p for p in p_value_list_simulated_aln_clock if p != None], [p for p in p_value_list_observed_aln_clock if p != None])
fig.update_layout(title='Test of clock violation - P-value distribution', 
                   xaxis_title='Uniform Quantiles', 
                   yaxis_title='p-value',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 20},  
                    xaxis_title_font={'size': 20}, 
                    width=None)
fig.show()

In [29]:
from collections import defaultdict
# Dictionary to store the count of values < 0.05 for each gene
count_less_than_005_clock = defaultdict(int)
# Dictionary to store the total count of entries for each gene
total_count = defaultdict(int)

for key, value in p_value_dict_observed_aln_clock.items():
    gene = key.split('_')[0]
    total_count[gene] += 1
    if value < 0.05:
        count_less_than_005_clock[gene] += 1

# Calculate the proportion of values < 0.05 for each gene
proportion_less_than_005_clock = {gene: count_less_than_005_clock[gene] / total for gene, total in total_count.items()}

count_gene_clock = len([x for x in proportion_less_than_005_clock.values() if x > 0.49])

count_gene_clock #73%, 43.8%

126

In [133]:
from collections import defaultdict
p_value_dict_by_gene_clock = {}
count_less_than_005_clock = defaultdict(int)
# Dictionary to store the total count of entries for each gene
total_count_clock = defaultdict(int)

for key, value in p_value_dict_observed_aln_clock.items():
    gene = key.split('_')[0]
    if gene not in p_value_dict_by_gene_clock:
        p_value_dict_by_gene_clock[gene] = []
        p_value_dict_by_gene_clock[gene].append(value)
    else:
        p_value_dict_by_gene_clock[gene].append(value)


estimated_non_null_prop_clock = {}
for gene, p_values in p_value_dict_by_gene_clock.items():
    try:
        estimated_non_null_prop_clock[gene] = 1-estimate_freq_null(np.array(p_values))
    except IndexError as e:
        print(gene)

### Intercept analysis

In [137]:
proportion_less_than_005_clock_filtered = {gene: proportion_less_than_005_clock[gene] for gene in proportion_less_than_005_non.keys()}

In [176]:
fig_0 = px.scatter(x = list(proportion_less_than_005_non.values()), y = list(proportion_less_than_005_clock.values()), labels={'x':'Clock Test Proportion of Significance', 'y':'Non-stationary Test Proportion of Significance'}, trendline="ols", title= None)
fig_0.update_layout(title='Proportion of Significance', 
                   xaxis_title='Non-stationarity Test', 
                   yaxis_title='Clock Test',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 20},  
                    xaxis_title_font={'size': 20}, 
                    width=None)
fig_0.update_layout(
    title={
        'text': 'Proportion of Significance',
        'y':0.95,
        'x':0.5,
        'yanchor': 'top',
        'font': dict(size=18, family='Arial', color='black')
    },
    xaxis_title='Non-stationarity Test',
    yaxis_title='Clock Test',
    xaxis=dict(
        title_font=dict(size=20, family='Arial', color='black'),
        tickfont=dict(size=14),
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        title_font=dict(size=20, family='Arial', color='black'),
        tickfont=dict(size=14),
        gridcolor='lightgrey'
    ),
    legend_title_text='Gene',
    legend=dict(
        title_font=dict(size=16),
        font=dict(size=14),
        bgcolor='rgba(255,255,255,0)',
        bordercolor='rgba(255,255,255,0)'
    ),
    margin=dict(l=80, r=80, t=100, b=80),
    autosize=True
)

# Customize the markers
fig_0.update_traces(
    marker=dict(
        size=10,
        opacity=0.8,
        line=dict(width=1, color='DarkSlateGrey')
    ),
    selector=dict(mode='markers')
)

# Customize the trendline
fig_0.update_traces(
    selector=dict(mode='lines'),
    line=dict(color='lightsalmon', width=2)
)

# Optional: Adjust hover template for better clarity
fig_0.update_traces(
    hovertemplate='<b>%{hovertext}</b><br>' +
                  'Non-stationarity Test: %{x}<br>' +
                  'Clock Test: %{y}<br>'
)

# Display the plot
fig_0.show()

In [135]:
estimated_non_null_prop_clock_filtered = {gene: estimated_non_null_prop_clock[gene] for gene in estimated_non_null_prop.keys()}

In [146]:
fig_0 = px.scatter(x = list(estimated_non_null_prop.values()), y = list(estimated_non_null_prop_clock_filtered.values()), labels={'x':'Clock Test Proportion of Significance', 'y':'Non-stationary Test Proportion of Significance'}, trendline="ols", title= None)
fig_0.update_layout(title='Proportion of Significance', 
                   xaxis_title='Non-stationarity Test', 
                   yaxis_title='Clock Test',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 20},  
                    xaxis_title_font={'size': 20}, 
                    width=None)
fig_0.show()

In [161]:
significant_triples_intercept = []
shared_triples = []
for gene in p_value_tested_dict.keys():
    if gene in p_value_dict_observed_aln_clock.keys():
        shared_triples.append(gene)
    if p_value_dict_observed_aln_clock[gene] < 0.05 and p_value_tested_dict[gene] < 0.05:
        significant_triples_intercept.append(gene)

In [162]:
len(significant_triples_intercept)/len(shared_triples)

0.392226148409894

In [150]:
i = 0
for key in list(p_value_tested_dict.keys()):
    if key in p_value_dict_observed_aln_clock.keys():
        i = i + 1

In [177]:
corr_list = {'ENSG00000137962': 0.5086889323577,
 'ENSG00000122482': 0.707100654789097,
 'ENSG00000116688': 0.19255296324868645,
 'ENSG00000122218': 0.03187622392335205,
 'ENSG00000054523': 0.03491225946607122,
 'ENSG00000030066': 0.31915613147914035,
 'ENSG00000177888': 0.3553500230545987,
 'ENSG00000117707': 0.07924470839043704,
 'ENSG00000122483': 0.15047452102022202,
 'ENSG00000138119': 0.4712037952032477,
 'ENSG00000107611': 0.2731737966610338,
 'ENSG00000149256': 0.2637244577227469,
 'ENSG00000085999': 0.5584775793921222,
 'ENSG00000187122': 0.43298106722994617,
 'ENSG00000135372': 0.3017929353998365,
 'ENSG00000197147': 0.6203589338624874,
 'ENSG00000148948': -0.0003109533310875593,
 'ENSG00000058668': 0.232437108708025,
 'ENSG00000116991': 0.48877073141629884,
 'ENSG00000165494': 0.11793556967637063,
 'ENSG00000066135': 0.15702377818724228,
 'ENSG00000174576': 0.2717970652102389,
 'ENSG00000148516': 0.6642292205924647,
 'ENSG00000163399': 0.41696608210499286,
 'ENSG00000129173': 0.5186525495153814,
 'ENSG00000110237': 0.18109352733818349,
 'ENSG00000185085': 0.2049356148532232,
 'ENSG00000162688': 0.8432122272856912,
 'ENSG00000133816': 0.2158127647684829,
 'ENSG00000151461': 0.007600217667331762,
 'ENSG00000117425': 0.21374375684542804,
 'ENSG00000260314': 0.6044174468125769,
 'ENSG00000197106': 0.1815673596789476,
 'ENSG00000143126': -0.018273276188891043,
 'ENSG00000095587': 0.17239263033316699,
 'ENSG00000058085': 0.1244956632402444,
 'ENSG00000187554': 0.5302333523050654,
 'ENSG00000143322': 0.2077034037005187,
 'ENSG00000265203': 0.19532227772437113,
 'ENSG00000110075': 0.30811978018211533,
 'ENSG00000016490': 0.6388443919018768,
 'ENSG00000171488': 0.7424080839486981,
 'ENSG00000148700': 0.34693435847753723,
 'ENSG00000142661': 0.23102071321802267,
 'ENSG00000196878': 0.2854810882548109,
 'ENSG00000117000': 0.06863061227122055,
 'ENSG00000143776': 0.6659785760169447,
 'ENSG00000116128': 0.19774972865274174,
 'ENSG00000117724': 0.15959286304767153,
 'ENSG00000048707': 0.47947242760662834,
 'ENSG00000118193': 0.7344148556269767,
 'ENSG00000166341': 0.4225699816958531,
 'ENSG00000117322': 0.5294490346333318,
 'ENSG00000110756': 0.7036010166725359,
 'ENSG00000065613': 0.8413486441042642,
 'ENSG00000138193': 0.47047239386087303,
 'ENSG00000009307': 0.5718839229663232,
 'ENSG00000129083': 0.6036338928356035,
 'ENSG00000095464': 0.4851577019870346,
 'ENSG00000148843': 0.24638922292796112,
 'ENSG00000166349': 0.20182663123839595,
 'ENSG00000117139': 0.5576170320526734,
 'ENSG00000143442': 0.1698371120405019,
 'ENSG00000081248': 0.14397022032396806,
 'ENSG00000196914': 0.5615057425160206,
 'ENSG00000052841': 0.5759381000494255,
 'ENSG00000138182': 0.31562919838781905,
 'ENSG00000158636': 0.591646227869138,
 'ENSG00000166106': 0.3995200508861711,
 'ENSG00000160695': -0.00506371746571319,
 'ENSG00000198216': 0.3303161575270735,
 'ENSG00000166507': 0.0175340993504133,
 'ENSG00000051009': 0.32900659614618866,
 'ENSG00000116539': 0.16303783004366573,
 'ENSG00000020129': 0.4525444405716277,
 'ENSG00000198952': 0.1648761166893308,
 'ENSG00000150347': 0.37298651018246837,
 'ENSG00000110497': 0.627733563954466,
 'ENSG00000198561': 0.7011257138969785,
 'ENSG00000118058': 0.6260987523345379,
 'ENSG00000134250': 0.407231132549167,
 'ENSG00000109927': 0.19820169776183352,
 'ENSG00000076356': 0.5330036640604715,
 'ENSG00000143702': 0.5291779629947241,
 'ENSG00000126705': 0.26994489768102964,
 'ENSG00000118200': 0.7101309711205078,
 'ENSG00000257315': 0.20157476111674583,
 'ENSG00000198730': 0.663290978667625,
 'ENSG00000137474': 0.3934509727613175,
 'ENSG00000065526': 0.33767938098646766,
 'ENSG00000047410': 0.30533425762989036,
 'ENSG00000204624': 0.16992871537673887,
 'ENSG00000095564': 0.48750536711034775,
 'ENSG00000126107': 0.3113982435729803,
 'ENSG00000214655': 0.23553360076915428,
 'ENSG00000107863': 0.5967844825507078,
 'ENSG00000167986': 0.44574260141970445,
 'ENSG00000175216': 0.21045607153241694,
 'ENSG00000075151': 0.23453174083702497,
 'ENSG00000133056': 0.2591954144315248,
 'ENSG00000152092': 0.1793157511844473,
 'ENSG00000117600': 0.6479831415961218,
 'ENSG00000110422': 0.5838928037582748,
 'ENSG00000116183': 0.3531468605141172,
 'ENSG00000154309': 0.4880061741793889,
 'ENSG00000158966': 0.5584461867426942,
 'ENSG00000169641': 0.23348820455595953,
 'ENSG00000121753': 0.2842249528009339,
 'ENSG00000133059': 0.021949449255250674,
 'ENSG00000107862': 0.3541037114044146,
 'ENSG00000162402': 0.35608864737235013,
 'ENSG00000075391': 0.297345906774692,
 'ENSG00000042781': 0.7220274161450632,
 'ENSG00000116147': 0.3316268122979195,
 'ENSG00000111642': 0.4221740550964627,
 'ENSG00000236287': 0.10718420156622406,
 'ENSG00000064309': 0.4618059576192681,
 'ENSG00000117114': 0.8356045568222351,
 'ENSG00000162711': 0.3393205002583897,
 'ENSG00000110799': 0.28377946395083814,
 'ENSG00000127124': 0.3501869195272494,
 'ENSG00000170242': 0.863527798261776,
 'ENSG00000173898': 0.11349610029300884,
 'ENSG00000143493': 0.7515487494210282,
 'ENSG00000173662': 0.07289592348576404,
 'ENSG00000162775': 0.1770098226387712,
 'ENSG00000023839': 0.1730462519936204,
 'ENSG00000135829': 0.45279987047942843,
 'ENSG00000177853': 0.3651018558274891,
 'ENSG00000176986': 0.5532034799760488}

In [181]:
corr_list_filtered = {gene: corr_list[gene] for gene in proportion_less_than_005_non.keys()}

fig_1 = px.scatter(x = list(proportion_less_than_005_non.values()), y = list(corr_list_filtered.values()), trendline="ols", title= None)
fig_1.update_layout(xaxis_title='Proportion of Significant Non-stationarity', 
                   yaxis_title='Correlation: Non-stationarity & Clock Violation',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=50, b=20),
                    autosize=True,
                    yaxis_title_font={'size': 20},  
                    xaxis_title_font={'size': 20}, 
                    width=None)
fig_1.update_layout(
    xaxis=dict(
        title_font=dict(size=20, family='Arial', color='black'),
        tickfont=dict(size=14),
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        title_font=dict(size=20, family='Arial', color='black'),
        tickfont=dict(size=14),
        gridcolor='lightgrey'
    ),
    legend_title_text='Gene',
    legend=dict(
        title_font=dict(size=16),
        font=dict(size=14),
        bgcolor='rgba(255,255,255,0)',
        bordercolor='rgba(255,255,255,0)'
    ),
    margin=dict(l=80, r=80, t=100, b=80),
    autosize=True
)

# Customize the markers
fig_1.update_traces(
    marker=dict(
        size=10,
        opacity=0.8,
        line=dict(width=1, color='DarkSlateGrey')
    ),
    selector=dict(mode='markers')
)

# Customize the trendline
fig_1.update_traces(
    selector=dict(mode='lines'),
    line=dict(color='lightsalmon', width=2)
)

# Optional: Adjust hover template for better clarity
fig_1.update_traces(
    hovertemplate='<b>%{hovertext}</b><br>' +
                  'Non-stationarity Test: %{x}<br>' +
                  'Clock Test: %{y}<br>'
)

# Display the plot
fig_1.show()
